In [1]:
# import pandas as pd

# df1 = pd.read_csv("https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets_raw/wp_log_peyton_manning.csv")
# df1.to_csv('c:/Users/Sofia/Documents/paper/Test/dataset/wp_log_peyton_manning.csv')


In [2]:
import os
import sys

import numpy as np
import pandas as pd
import random

import torch
print(f"available gpu: {torch.cuda.is_available()}")

import argparse

sys.path.insert(0, 'c:/Users/Sofia/Documents/paper/Test/TimesNet_code')
from print_args import print_args
from exp_long_term_forecasting import Exp_Long_Term_Forecast

fix_seed = 2024
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

available gpu: True


In [3]:
args = argparse.Namespace(
    # basic config
    task_name = 'long_term_forecast'
    , is_training = 1
    , model_id = 'pm_ver1'
    , model = 'TimesNet'

    # data loader
    , data = 'custom'
    , root_path = 'c:/Users/Sofia/Documents/paper/Test/dataset/' ### 데이터 확인 필요
    , data_path = 'wp_log_peyton_manning.csv' ### 데이터 확인 필요
    , features = 'S' ### M: 다변량 -> 다변량, S: 일변량 -> 일변량, MS: 다변량 -> 일변량
    , target = 'y'
    , date_col = 'ds'
    , train_test_split_date = ['2012-11-09', '2014-06-16']
    , train_test_split_rate = None
    , freq = 'd'
    , checkpoints = './checkpoints/'

    # forecasting task
    , seq_len = 96
    , label_len = 48
    , pred_len = 96
    , seasonal_patterns = 'Monthly'
    , inverse = False

    # inputation task ### 삭제?
    , mask_rate = 0.25 

    # anomaly detection task ### 삭제?
    , anomaly_ratio = 0.25 

    # model define
    , top_k = 5
    , num_kernels = 6 # N: Transformer 주요 파라미터
    , enc_in = 1 # 사용 변수 수
    , dec_in = 1 # 사용 변수 수
    , c_out = 1 # 사용 변수 수
    , d_model = 512 # Transformer 주요 파라미터
    , n_heads = 8 # h: Transformer 주요 파라미터
    , e_layers = 2
    , d_layers = 1
    , d_ff = 2048 # Transformer 주요 파라미터
    , moving_avg = 25
    , factor = 1
    , distil = True
    , dropout = 0.1
    , embed = 'timeF'
    , activation = 'gelu'
    , output_attention = False ### default 없음 추가로 arg 지정 없으면 false

    # optimization
    , num_workers = 10
    , itr = 1
    , train_epochs = 10
    , batch_size = 32 # 32
    , patience = 3 #
    , learning_rate = 0.000001
    , des = 'test'
    , loss = 'MSE'
    , lradj = 'type1'
    , use_amp = False

    # GPU
    , use_gpu = False
    , gpu = 0
    , use_multi_gpu = False
    , devices = '0'

    # de-stationary projector params
    , p_hidden_dims = [128, 128]
    , p_hidden_layers = 2
)

args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

print('Args in experiment:')
print_args(args)

Exp = Exp_Long_Term_Forecast

if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        exp = Exp(args)  # set experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.task_name,
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)
        torch.cuda.empty_cache()
else:
    ii = 0
    setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.task_name,
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.distil,
        args.des, ii)

    exp = Exp(args)  # set experiments
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting, test=1)
    torch.cuda.empty_cache()

    # (1743, 581, 581)
    # (1552, 486, 486)
    # -191, -95, -95

#PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           pm_ver1             Model:              TimesNet            

Data Loader
  Data:               custom              Root Path:          c:/Users/Sofia/Documents/paper/Test/dataset/
  Data Path:          wp_log_peyton_manning.csvFeatures:           S                   
  Target:             y                   Freq:               d                   
  Checkpoints:        ./checkpoints/      

Forecasting Task
  Seq Len:            96                  Label Len:          48                  
  Pred Len:           96                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              5                   Num Kernels:        6                   
  Enc In:             1                   Dec In:             1                   
  C Out:              1                   d model:      

AttributeError: 'DataFrame' object has no attribute 'date'

In [4]:
args = argparse.Namespace(
    # basic config
    task_name = 'long_term_forecast'
    , is_training = 1
    , model_id = 'pm_ver2'
    , model = 'TimesNet'

    # data loader
    , data = 'custom'
    , root_path = 'c:/Users/Sofia/Documents/paper/Test/dataset/' ### 데이터 확인 필요
    , data_path = 'wp_log_peyton_manning.csv' ### 데이터 확인 필요
    , features = 'S' ### M: 다변량 -> 다변량, S: 일변량 -> 일변량, MS: 다변량 -> 일변량
    , target = 'y'
    , freq = 'd'
    , checkpoints = './checkpoints/'

    # forecasting task
    , seq_len = 96
    , label_len = 48
    , pred_len = 96
    , seasonal_patterns = 'Monthly'
    , inverse = False

    # inputation task ### 삭제?
    , mask_rate = 0.25 

    # anomaly detection task ### 삭제?
    , anomaly_ratio = 0.25 

    # model define
    , top_k = 10
    , num_kernels = 6 # N: Transformer 주요 파라미터
    , enc_in = 1 # 사용 변수 수
    , dec_in = 1 # 사용 변수 수
    , c_out = 1 # 사용 변수 수
    , d_model = 512 # Transformer 주요 파라미터
    , n_heads = 8 # h: Transformer 주요 파라미터
    , e_layers = 2
    , d_layers = 1
    , d_ff = 2048 # Transformer 주요 파라미터
    , moving_avg = 25
    , factor = 1
    , distil = True
    , dropout = 0.1
    , embed = 'timeF'
    , activation = 'gelu'
    , output_attention = False ### default 없음 추가로 arg 지정 없으면 false

    # optimization
    , num_workers = 10
    , itr = 1
    , train_epochs = 1
    , batch_size = 32 # 32
    , patience = 3 #
    , learning_rate = 0.000001
    , des = 'test'
    , loss = 'MSE'
    , lradj = 'type1'
    , use_amp = False

    # GPU
    , use_gpu = False
    , gpu = 0
    , use_multi_gpu = False
    , devices = '0'

    # de-stationary projector params
    , p_hidden_dims = [128, 128]
    , p_hidden_layers = 2
)

args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

print('Args in experiment:')
print_args(args)

Exp = Exp_Long_Term_Forecast

if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        exp = Exp(args)  # set experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.task_name,
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)
        torch.cuda.empty_cache()
else:
    ii = 0
    setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.task_name,
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.distil,
        args.des, ii)

    exp = Exp(args)  # set experiments
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting, test=1)
    torch.cuda.empty_cache()

    # (1743, 581, 581)
    # (1552, 486, 486)
    # -191, -95, -95

#PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           pm_ver2             Model:              TimesNet            

Data Loader
  Data:               custom              Root Path:          c:/Users/Sofia/Documents/paper/Test/dataset/
  Data Path:          wp_log_peyton_manning.csvFeatures:           S                   
  Target:             y                   Freq:               d                   
  Checkpoints:        ./checkpoints/      

Forecasting Task
  Seq Len:            96                  Label Len:          48                  
  Pred Len:           96                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              10                  Num Kernels:        6                   
  Enc In:             1                   Dec In:             1                   
  C Out:              1                   d model:      

In [3]:
args = argparse.Namespace(
    # basic config
    task_name = 'long_term_forecast'
    , is_training = 1
    , model_id = 'pm_ver3'
    , model = 'TimesNet'

    # data loader
    , data = 'custom'
    , root_path = 'c:/Users/Sofia/Documents/paper/Test/dataset/' ### 데이터 확인 필요
    , data_path = 'wp_log_peyton_manning.csv' ### 데이터 확인 필요
    , features = 'S' ### M: 다변량 -> 다변량, S: 일변량 -> 일변량, MS: 다변량 -> 일변량
    , target = 'y'
    , freq = 'd'
    , checkpoints = './checkpoints/'

    # forecasting task
    , seq_len = 96
    , label_len = 48
    , pred_len = 96
    , seasonal_patterns = 'Monthly'
    , inverse = False

    # inputation task ### 삭제?
    , mask_rate = 0.25 

    # anomaly detection task ### 삭제?
    , anomaly_ratio = 0.25 

    # model define
    , top_k = 15
    , num_kernels = 6 # N: Transformer 주요 파라미터
    , enc_in = 1 # 사용 변수 수
    , dec_in = 1 # 사용 변수 수
    , c_out = 1 # 사용 변수 수
    , d_model = 512 # Transformer 주요 파라미터
    , n_heads = 8 # h: Transformer 주요 파라미터
    , e_layers = 2
    , d_layers = 1
    , d_ff = 2048 # Transformer 주요 파라미터
    , moving_avg = 25
    , factor = 1
    , distil = True
    , dropout = 0.1
    , embed = 'timeF'
    , activation = 'gelu'
    , output_attention = False ### default 없음 추가로 arg 지정 없으면 false

    # optimization
    , num_workers = 10
    , itr = 1
    , train_epochs = 1
    , batch_size = 32 # 32
    , patience = 3 #
    , learning_rate = 0.000001
    , des = 'test'
    , loss = 'MSE'
    , lradj = 'type1'
    , use_amp = False

    # GPU
    , use_gpu = False
    , gpu = 0
    , use_multi_gpu = False
    , devices = '0'

    # de-stationary projector params
    , p_hidden_dims = [128, 128]
    , p_hidden_layers = 2
)

args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

print('Args in experiment:')
print_args(args)

Exp = Exp_Long_Term_Forecast

if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        exp = Exp(args)  # set experiments
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.task_name,
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)
        torch.cuda.empty_cache()
else:
    ii = 0
    setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.task_name,
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.distil,
        args.des, ii)

    exp = Exp(args)  # set experiments
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting, test=1)
    torch.cuda.empty_cache()

    # (1743, 581, 581)
    # (1552, 486, 486)
    # -191, -95, -95

#PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           pm_ver3             Model:              TimesNet            

Data Loader
  Data:               custom              Root Path:          c:/Users/Sofia/Documents/paper/Test/dataset/
  Data Path:          wp_log_peyton_manning.csvFeatures:           S                   
  Target:             y                   Freq:               d                   
  Checkpoints:        ./checkpoints/      

Forecasting Task
  Seq Len:            96                  Label Len:          48                  
  Pred Len:           96                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              15                  Num Kernels:        6                   
  Enc In:             1                   Dec In:             1                   
  C Out:              1                   d model:      